In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from absbox.local.util import guess_pool_flow_header

In [59]:
#### from absbox.local.china import SPV
from absbox import API
localAPI = API("http://localhost:8081",lang='english',check=False)


assets = [
            ["FixedAsset"
              ,{"start":"2023-11-01","originBalance":1200,"originTerm":2
               ,"residual":200,"period":"Monthly","amortize":"Straight"
               ,"capacity":("Fixed",100)}
              ,{"balance":489.90,"remainTerm":2}]
    ,            ["FixedAsset"
              ,{"start":"2024-01-01","originBalance":1200,"originTerm":2
               ,"residual":200,"period":"Monthly","amortize":"Straight"
               ,"capacity":("Fixed",100)}
              ,{"balance":489.90,"remainTerm":2}]  
    ] 
    

myAssump = ("Pool"
            ,("Fixed",[["2022-01-01",0.1]]
                     ,[["2022-01-01",400]
                      ,["2024-09-01",300]])
            ,None
            ,None)

p = localAPI.runAsset("2021-04-01"
                      ,assets
                      ,poolAssump=myAssump
                      ,read=True)
p[0]

✅Connected, local lib:0.20.2, server:0.22.4

,Balance,Depreciation,CumuDepreciation,Unit,Cash
Date,,,,,
2023-12-01,1900,500,500,10,4000
2024-01-01,1400,500,1000,10,4000
2024-02-01,900,500,500,10,4000
2024-03-01,400,500,1000,10,4000


In [201]:
#### from absbox.local.china import SPV
from absbox import API
localAPI = API("http://localhost:8081",lang='english',check=False)


assets = [
            ["FixedAsset"
              ,{"start":"2023-11-01","originBalance":1200,"originTerm":2
               ,"residual":200,"period":"Monthly","amortize":"DecliningBalance"
               ,"capacity":("Fixed",100)}
              ,{"balance":1200,"remainTerm":2}]  
    ] 
    

myAssump = ("Pool"
            ,("Fixed",[["2022-01-01",0.1]]
                     ,[["2022-01-01",400]
                      ,["2024-09-01",300]])
            ,None
            ,None)

p = localAPI.runAsset("2021-04-01"
                      ,assets
                      ,poolAssump=myAssump
                      ,read=True)
p[0]

⠋ Connecting engine server -> http://localhost:8081
✅Connected, local lib:0.20.2, server:0.22.3
⠋


,Balance,Depreciation,CumuDepreciation,Unit,Cash
Date,,,,,
2023-12-01,700,500,500,10,4000
2024-01-01,200,500,1000,10,4000


In [45]:
from absbox.local.generic import Generic
from absbox import API
localAPI = API("http://localhost:8081",lang='english',check=False)

assets = [["FixedAsset"
            ,{"start":"2023-11-01","originBalance":15000,"originTerm":240
              ,"residual":1000,"period":"Monthly","amortize":"Straight"
              ,"capacity":("Fixed",20*20*2)}
            ,{"remainTerm":240}]] 


exps = (("maintenance",{"type":{"recurFee":["YearFirst",200]}})
        ,("maintenance2",{"type":{"recurFee":["YearFirst",200],"feeStart":"2033-11-01"}})
        ,("insurance",{"type":{"recurFee":["MonthFirst",20]}}))

waterfall = {"amortizing":[
                 ["payFee","acc01",['maintenance','maintenance2','insurance']]
                ,["accrueAndPayInt","acc01",["A"]]
                ,["payPrin","acc01",["A"]]
                ,["payPrin","acc01",["EQ"]
                  ,{"limit":{"formula":("floorWithZero"
                               ,("substract"
                                 ,("accountBalance","acc01"),("constant",1045)))}}]
               ]
             ,"cleanUp":[
                 ["sellAsset",["Current|Defaulted",1.0,0],"acc01"]
                 ,["payPrin","acc01",["EQ"]]
                 ,["payIntResidual","acc01","EQ"]
             ]}


solarPanel = Generic(
    "TEST01"
    ,{"cutoff":"2024-01-01","closing":"2024-01-01","firstPay":"2024-02-01"
     ,"payFreq":"MonthEnd","poolFreq":"MonthEnd","stated":"2050-01-01"}
    ,{'assets':assets}
    ,(("acc01",{"balance":0}),)
    ,(("A",{"balance":9_000
           ,"rate":0.05
           ,"originBalance":9_000
           ,"originRate":0.00
           ,"startDate":"2024-01-01"
           ,"rateType":{"Fixed":0.05}
           ,"bondType":{"Sequential":None}})
      ,("EQ",{"balance":7_000
           ,"rate":0.0
           ,"originBalance":7_000
           ,"originRate":0.00
           ,"startDate":"2024-01-01"
           ,"rateType":{"Fixed":0.00}
           ,"bondType":{"Equity":None}}),
     )
    ,exps
    ,waterfall
    ,[["CollectedCash","acc01"]]
    ,None
    ,None
    ,None
    ,None
    ,("PreClosing","Amortizing")
    )

myAssump = ("Pool"
            ,("Fixed",[["2024-01-01",0.3]
                      ,["2025-01-01",0.25]
                      ,["2026-01-01",0.2]]
                     ,[["2024-01-01",0.9]
                      ,["2025-01-01",0.85]
                      ,["2026-01-01",0.8]])
            ,None
            ,None)

p = localAPI.run(solarPanel,poolAssump=myAssump
                       ,runAssump=[("call",{"afterDate":"2044-01-01"})
                                  ,("report",{"dates":"MonthEnd"})]
                       ,read=True)

⠙

In [46]:
p['result']['bonds']

,Balance Defaults,Interest Defaults,Original Balance,Total
A,0,0,9000,0
EQ,0,0,7000,0


In [47]:
p['pool']['flow']

,Balance,Depreciation,CumuDepreciation,Unit,Cash
Date,,,,,
2024-01-01,14883.34,0.00,0.00,0,0
2024-01-01,14883.34,58.33,116.66,240,216
2024-02-29,14825.01,58.33,174.99,240,216
2024-03-31,14766.68,58.33,233.32,240,216
2024-04-30,14708.35,58.33,291.65,240,216
...,...,...,...,...,...
2043-07-31,1234.12,58.33,13765.88,160,128
2043-08-31,1175.79,58.33,13824.21,160,128
2043-09-30,1117.46,58.33,13882.54,160,128


In [48]:
p['result']['report']['cash']

inflow                      \
                      PoolInflow CollectedCash LiquidationProceeds   
startDate  endDate                                                   
2024-01-31 2024-02-29                    216.0                 NaN   
2024-02-29 2024-03-31                    216.0                 NaN   
2024-03-31 2024-04-30                    216.0                 NaN   
2024-04-30 2024-05-31                    216.0                 NaN   
2024-05-31 2024-06-30                    216.0                 NaN   
...                                        ...                 ...   
2043-08-31 2043-09-30                    128.0                 NaN   
2043-09-30 2043-10-31                    128.0                 NaN   
2043-10-31 2043-11-30                    128.0                 NaN   
2043-11-30 2043-12-31                      NaN                 NaN   
2043-12-31 2044-01-31                      NaN              1000.8   

                           outflow                \
                      PayInt ["A"] PayPrin ["A"]   
startDate  endDate                                 
2024-01-31 2024-02-29       -72.04       -339.96   
2024-02-29 2024-03-31       -36.77       -159.23   
2024-03-31 2024-04-30       -34.93       -161.07   
2024-04-30 2024-05-31       -35.41       -160.59   
2024-05-31 2024-06-30       -33.61       -162.39   
...                            ...           ...   
2043-08-31 2043-09-30         0.00          0.00   
2043-09-30 2043-10-31         0.00          0.00   
2043-10-31 2043-11-30         0.00          0.00   
2043-11-30 2043-12-31         0.00          0.00   
2043-12-31 2044-01-31         0.00          0.00   

                                                                            \
                      SeqPayFee ["maintenance","maintenance2","insurance"]   
startDate  endDate                                                           
2024-01-31 2024-02-29                                              -20.0     
2024-02-29 2024-03-31                                              -20.0     
2024-03-31 2024-04-30                                              -20.0     
2024-04-30 2024-05-31                                              -20.0     
2024-05-31 2024-06-30                                              -20.0     
...                                                                  ...     
2043-08-31 2043-09-30                                              -20.0     
2043-09-30 2043-10-31                                              -20.0     
2043-10-31 2043-11-30                                              -20.0     
2043-11-30 2043-12-31                                              -20.0     
2043-12-31 2044-01-31                                             -420.0     

                                                                                                                                   \
                      TxnComments [PayPrin ["EQ"],UsingDS (FloorWithZero (Substract [AccBalance ["acc01"],Constant (1045 % 1)]))]   
startDate  endDate                                                                                                                  
2024-01-31 2024-02-29                                                0.0                                                            
2024-02-29 2024-03-31                                                0.0                                                            
2024-03-31 2024-04-30                                                0.0                                                            
2024-04-30 2024-05-31                                                0.0                                                            
2024-05-31 2024-06-30                                                0.0                                                            
...                                                                  ...                                                            
2043-08-31 2043-09-30                   

In [49]:
p['bonds']['EQ']

,balance,interest,principal,rate,cash,memo
date,,,,,,
2024-02-01,7000.00,0.00,0.00,0,0.00,<PayPrin:EQ>
2024-02-29,7000.00,0.00,0.00,0,0.00,<PayPrin:EQ>
2024-03-31,7000.00,0.00,0.00,0,0.00,<PayPrin:EQ>
2024-04-30,7000.00,0.00,0.00,0,0.00,<PayPrin:EQ>
2024-05-31,7000.00,0.00,0.00,0,0.00,<PayPrin:EQ>
...,...,...,...,...,...,...
2043-03-31,155.79,0.00,0.00,0,0.00,<PayPrin:EQ>
2043-04-30,123.79,0.00,32.00,0,32.00,<PayPrin:EQ>
2043-05-31,15.79,0.00,108.00,0,108.00,<PayPrin:EQ>


In [50]:
from absbox.local.util import irr
irr(p['bonds']['EQ'],init=('2024-01-01',-7_000))

0.016930937065270275

In [51]:
scenarioMap = {
    "base":("Pool"
            ,("Fixed",[["2024-01-01",0.3]
                      ,["2025-01-01",0.25]
                      ,["2026-01-01",0.2]]
                     ,[["2024-01-01",0.9]
                      ,["2025-01-01",0.85]
                      ,["2026-01-01",0.8]])
            ,None
            ,None)
    ,"lowUtil" :("Pool"
                ,("Fixed",[["2024-01-01",0.3]
                          ,["2025-01-01",0.25]
                          ,["2026-01-01",0.2]]
                         ,[["2024-01-01",0.85]
                          ,["2025-01-01",0.80]
                          ,["2026-01-01",0.75]])
                ,None
                ,None)
   ,"lowPrice" : ("Pool"
                ,("Fixed",[["2024-01-01",0.3]
                          ,["2025-01-01",0.225]
                          ,["2026-01-01",0.19]]
                         ,[["2024-01-01",0.9]
                          ,["2025-01-01",0.85]
                          ,["2026-01-01",0.8]])
                ,None
                ,None)
}
p = localAPI.run(solarPanel,poolAssump=scenarioMap
                       ,runAssump=[("call",{"afterDate":"2044-01-01"})
                                  ,("report",{"dates":"MonthEnd"})]
                       ,read=True)

⠦

In [53]:
from absbox.local.util import irr
{k:irr(v['bonds']['EQ'],init=('2024-01-01',-7000)) 
 for k,v in p.items()}

{'base': 0.016930937065270275,
 'lowPrice': 0.0028372850153230164,
 'lowUtil': -0.0013152392627518972}